In [1]:
!pip install emoji

In [2]:
import re
from sklearn.model_selection import train_test_split
import emoji

In [3]:
import pandas as pd
data = pd.read_csv("reduced_data.csv")

In [ ]:
def preprocess_tweet(tweet):
    # # Convert emojis to text
    tweet = emoji.demojize(tweet, delimiters=(":", ":"))
    #strip starting with '@'
    tweet = re.sub(r'@\w+', '', tweet)
    #strip urls
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)
    tweet = tweet.lower()
    return tweet


data['cleaned_tweet'] = data['tweet'].apply(preprocess_tweet)



In [5]:
train_data, temp_data = train_test_split(data, test_size=0.3, stratify=data['class'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['class'], random_state=42)

In [6]:
import torch

if not torch.cuda.is_available():
  print("No GPU detected. Please add a GPU, for example, via Colab's UI.")

else:
  device_name = torch.cuda.get_device_name()
  n_gpu = torch.cuda.device_count()
  print("Found device: {}, n_gpu: {}".format(device_name, n_gpu))

Found device: Tesla T4, n_gpu: 1


In [7]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# load tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# model_name = "cardiffnlp/twitter-roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#Preparing Datasets

class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }


In [10]:

#prepare datasets

train_dataset = HateSpeechDataset(train_data["cleaned_tweet"], train_data["class"], tokenizer)
val_dataset = HateSpeechDataset(val_data["cleaned_tweet"], val_data["class"], tokenizer)
test_dataset = HateSpeechDataset(test_data["cleaned_tweet"], test_data["class"], tokenizer)


In [11]:
#define training arguments
training_args = TrainingArguments(
    output_dir='./results',         # output directory
    num_train_epochs=3,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    evaluation_strategy="epoch",    # evaluation occurs after each epoch
    logging_dir='./logs',           # directory for storing logs
    logging_strategy="epoch",       # logging occurs after each epoch
    log_level="error",            # set logging level
    optim="adamw_torch",            # use pytorch's adamw implementation
    report_to="none",
    learning_rate=2e-5

)

def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate(test_dataset)
print(results)



<ipython-input-12-72cec41ac2a2>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.778500,0.524198,0.813333
2,0.414000,0.545313,0.786667
3,0.294000,0.499201,0.802222


{'eval_loss': 0.5537990927696228, 'eval_accuracy': 0.8177777528762817, 'eval_runtime': 3.0957, 'eval_samples_per_second': 145.364, 'eval_steps_per_second': 2.584, 'epoch': 3.0}


In [13]:
from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1).numpy()

print(classification_report(test_data["class"], preds, target_names=["Hate Speech", "Offensive Language", "Neither"]))


                    precision    recall  f1-score   support

       Hate Speech       0.77      0.76      0.76       150
Offensive Language       0.83      0.76      0.79       150
           Neither       0.86      0.93      0.89       150

          accuracy                           0.82       450
         macro avg       0.82      0.82      0.82       450
      weighted avg       0.82      0.82      0.82       450

